In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

import torchvision # アーキテクチャが保存されている
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor, Lambda
import matplotlib.pyplot as plt

from PIL import Image
import pandas as pd
import numpy as np
import re
import random

device = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu") # GPUがあるかどうかをチェック

/usr/local/lib/python3.12/dist-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /opt/pytorch/pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
training_data = datasets.FashionMNIST(
    root="./../../data/normalization",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="./../../data/normalization",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [3]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size) # PytorchはDataLoderでミニバッチ学習用のデータ準備する。
test_dataloader = DataLoader(test_data, batch_size=batch_size) # PytorchはDataLoderでミニバッチ学習用のデータ準備する

In [4]:
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10) # FashionMNISTは10クラス分類
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [5]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train() # Trainする際に使用するモード
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0: # 100回に1回、lossとデータ数を表示
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval() # Predictする際に使用するモード
    test_loss, correct = 0, 0

    with torch.no_grad(): # テンソル勾配の計算をさせないモード(Predictなので勾配計算は不要)
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size

    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

    return test_loss, correct

In [6]:
model = MyModel()

loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params = model.parameters(), lr=0.05)

epochs = 20
epoch_loss = []
epoch_acc = []
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_function, optimizer)
    test_return = test_loop(test_dataloader, model, loss_function)
    epoch_loss.append(test_return[0])
    epoch_acc.append(test_return[1])
print("Done!")

Epoch 1
-------------------------------
loss: 2.294697  [    0/60000]
loss: 1.142566  [ 6400/60000]
loss: 0.716798  [12800/60000]
loss: 0.786623  [19200/60000]
loss: 0.676206  [25600/60000]
loss: 0.573911  [32000/60000]
loss: 0.628836  [38400/60000]
loss: 0.655481  [44800/60000]
loss: 0.673701  [51200/60000]
loss: 0.483299  [57600/60000]
Test Error: 
 Accuracy: 78.1%, Avg loss: 0.607873 

Epoch 2
-------------------------------
loss: 0.472452  [    0/60000]
loss: 0.509028  [ 6400/60000]
loss: 0.384396  [12800/60000]
loss: 0.530894  [19200/60000]
loss: 0.514582  [25600/60000]
loss: 0.460757  [32000/60000]
loss: 0.507159  [38400/60000]
loss: 0.578125  [44800/60000]
loss: 0.590402  [51200/60000]
loss: 0.432264  [57600/60000]
Test Error: 
 Accuracy: 80.4%, Avg loss: 0.530600 

Epoch 3
-------------------------------
loss: 0.376970  [    0/60000]
loss: 0.439785  [ 6400/60000]
loss: 0.321300  [12800/60000]
loss: 0.449206  [19200/60000]
loss: 0.414112  [25600/60000]
loss: 0.431947  [32000/600